In [1]:
from main_extractor import *
from utils.text_feature import *
from utils.dict_lib import *

fs = [TextFeature("rank", "mean_key"), TextFeature("words", "select_keys_proportion", args=("я", "ты")), 
      TextFeature("parts", "select_keys_proportion", args=("VERB",)), TextFeature("len_words", "mean_key", args=(16,)),
      TextFeature("homogeneity", "mean_key", args=(100,))]

d = text2fеatures("1813\\3", fs)
print(d)

d = important_feature("words", n_top=30, rang=(2, 5))
print(d)

print(count_features(fs))
print(get_names(fs))

[0.1004481418852913, 0.03561643835616438, 0.03561643835616438, 0.18904109589041096, 0.292722602739726, 0.13]
{'и': 4, 'в': 4, 'я': 4, 'не': 4, 'с': 4, 'ты': 4, 'на': 4, 'он': 4, 'мой': 4, 'но': 4, 'что': 4, 'быть': 4, 'она': 4, 'друг': 2, 'как': 4, 'к': 4, 'любовь': 2, 'о': 2, 'за': 4, 'свой': 4, 'вы': 4, 'твой': 3, 'мы': 4, 'же': 4, 'где': 2, 'день': 2, 'они': 3, 'то': 3, 'а': 2, 'да': 2, 'так': 2, 'весь': 2, 'нет': 2}
33
{'ный': 4, 'ост': 4, 'ной': 4, 'енн': 4, 'ень': 4, 'ста': 4, 'ест': 4, 'стр': 4, 'аст': 4, 'сто': 4, 'дру': 3, 'сти': 4, 'нны': 3, 'рас': 3, 'при': 4, 'пре': 3, 'про': 4, 'ого': 4, 'оро': 4, 'нов': 3, 'под': 3, 'али': 4, 'тво': 4, 'вет': 3, 'нес': 3, 'мне': 4, 'его': 3, 'как': 3, 'раз': 3, 'мен': 3, 'ово': 3, 'ать': 3}
6
['rank', 'words/я', 'words/ты', 'parts/VERB', 'len_words', 'homogeneity']


In [1]:
from models.data import *
from periods import *

fs = [TextFeature("homogeneity", "mean_key", args=(100,)), 
      TextFeature("rank", "mean_key"),
      TextFeature("len_words", "mean_key", args=(25,)),
      TextFeature("count_puncts_in_sentence", "mean_key", args=(25,)),
      TextFeature("count_words_in_sentence", "mean_key", args=(100,)),
      TextFeature("words", "select_keys_proportion", args=("я", "что")),
      TextFeature("puncts", "select_keys_proportion", args=("!",)),
      TextFeature("parts", "select_keys_proportion", args=("ADJ", "PRON", "PART")), 
      TextFeature("rels", "select_keys_proportion", args=("conj", "advmod", "amod", "case", "obl", "obj", "nmod")),
      TextFeature("n_grams_letter", "select_keys_proportion", args=("ный", "ста", "его", "ень", "что"))]
print(count_features(fs))

per1 = periods[-1]
per2 = sum(periods[:-1], [])
pers = [per1, per2]

data = get_data(pers, fs, train_part=0.8, val_part=0.1, not_null_bord=14)
print(data.size())

23
709


In [ ]:
from models.fit import fit
from models.models import get_dense_model
from utils.text_feature import count_features
from models.data import *

data = load_data('data4-1')
epochs = 250
model = get_dense_model(count_features(fs), [16, 12, 8, 4])
model.summary()
fit(model, data, epochs)

In [4]:
from models.data import *

data.save('data4-1')

In [ ]:
model.save(root + "models\\saved_models\\model4-1.keras")

In [ ]:
from keras import models
from models.data import *
from models.fit import show_metrics

#data = load_data('data4-1')
#loaded_model = models.load_model(root + "models\\saved_models\\model4-1.keras")
loaded_model = model
loaded_model.summary()
predicts = loaded_model.predict(data.x_val+data.x_test)
preds = [1 - p for p in predicts]
labels = [1 - y for y in data.y_val+data.y_test]
show_metrics(predicts, data.y_val+data.y_test)
#show_metrics(preds, labels)

In [ ]:
from keras import models
from models.fit import shap_explain
from models.data import *

d = load_data('data4-1')
loaded_model = models.load_model(root + "models\\saved_models\\model4-1.keras")
loaded_model.summary()
shap_explain(d, get_names(fs), loaded_model.predict, 'model4-1')

In [ ]:
def parse(d: dict) -> str:
    res = ''
    for key in d:
        if d[key] > 0:
            res += f'[{key}]*{d[key]} + '
    return res[:-3]

In [ ]:
from models.ensemble import *

models = ["model1", "model2", "model4", "model4-1", "model5"]
    
ensemble_thresholds_test(models, 0)

In [6]:
from utils.ensemble_metadata import get
from utils.text_feature import *
from utils.dict_lib import dict_sort

models = ["model1", "model2", "model4", "model4-1", "model5"]
datas = ["data1", "data2", "data4", "data4-1", "data5"]
dfs = dict()

fs = get(models, "features")
for f in fs:
    sf = get_names(f)
    for s in sf:
        dfs[s] = dfs.get(s, 0) + 1
print(dict_sort(dfs))

{'homogeneity': 5, 'rank': 5, 'parts/PRON': 5, 'rels/advmod': 5, 'rels/amod': 5, 'len_words': 4, 'count_words_in_sentence': 4, 'words/что': 4, 'puncts/!': 4, 'parts/ADJ': 4, 'rels/conj': 4, 'n_grams_letter/его': 4, 'count_puncts_in_sentence': 3, 'words/я': 3, 'rels/obl': 3, 'rels/nmod': 3, 'n_grams_letter/ста': 3, 'n_grams_letter/что': 3, 'words/любовь': 2, 'words/друг': 2, 'words/они': 2, 'parts/ADV': 2, 'n_grams_letter/енн': 2, 'n_grams_all_symb/ой ': 2, 'parts/PART': 2, 'rels/case': 2, 'rels/obj': 2, 'n_grams_letter/ный': 2, 'n_grams_letter/ень': 2, 'words/ты': 1, 'words/она': 1, 'parts/ADP': 1, 'n_grams_letter/ной': 1, 'words/вы': 1, 'words/мы': 1, 'words/он': 1, 'parts/PROPN': 1, 'rels/nsubj': 1, 'n_grams_all_symb/ не': 1, 'n_grams_all_symb/не ': 1, 'n_grams_all_symb/ мо': 1, 'puncts/?': 1, 'n_grams_all_symb/ на': 1, 'n_grams_all_symb/ по': 1, 'n_grams_letter/как': 1, 'n_grams_letter/люб': 1}


In [7]:
print(len(dfs))

46
